In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

    
val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

/*
val lr = new LogisticRegression()
    .setMaxIter(10)
    .setRegParam(0.001)
*/

// описываем модель RandomForest
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
val rf = new RandomForestClassifier()
  .setLabelCol("label")
  .setFeaturesCol("features")
  .setNumTrees(10)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, rf))


tokenizer = tok_b13ecf247519
hashingTF = hashingTF_6f4077563c32
rf = rfc_54dffeb558cf
pipeline = pipeline_d0f6df2ac549


pipeline_d0f6df2ac549

In [4]:
val model = pipeline.fit(raw_sentiment)

pipeline_d0f6df2ac549

model = pipeline_d0f6df2ac549


In [5]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [6]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_d0f6df2ac549


pipeline_d0f6df2ac549

In [7]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|[4.41571348674201...|[0.44157134867420...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|[5.38315316533973...|[0.53831531653397...|       0.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|[5.98890660175933...|[0.59889066017593...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[82,191,296...|[5.63521958939778...|[0.56352195893977...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[18,96,130,...|[5.33699915538184...|[0.53369991553818...|       0.0|
|    0|@

predictionsDF = [label: int, tweet: string ... 5 more fields]


[label: int, tweet: string ... 5 more fields]

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

+-------------------+
|  clean_probability|
+-------------------+
| 0.5584286513257988|
|0.46168468346602654|
|0.40110933982406627|
|0.43647804106022187|
|0.46630008446181526|
| 0.5127946991626604|
| 0.5127946991626604|
|  0.548059765268196|
| 0.5127946991626604|
| 0.5056272416067278|
| 0.5127946991626604|
|0.43647804106022187|
|  0.462528909677632|
| 0.5846642279075845|
| 0.4887831061811626|
| 0.5299836440186307|
| 0.5056272416067278|
| 0.5127946991626604|
| 0.5347594949965139|
| 0.5459298692463014|
+-------------------+
only showing top 20 rows

